In [249]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import datetime
import random
import re
from urllib.request import urlopen
from urllib.error import HTTPError
from urllib.error import URLError
import requests
import re
import py2neo as neo
import pandas as pd
import numpy as np
import random

In [250]:
Group=[[] for _ in range(10)]
teams=[[] for _ in range(10)]
country=[[] for _ in range(10)]

In [251]:
def creat_seasons_link(i):
    year=2021-i
    if year<=2007:
        year=year-1
    return ("https://www.uefa.com/uefachampionsleague/history/seasons/"+str(year)+"/")
    

In [252]:
def get_nation(link):
    link="https://www.uefa.com"+link
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find("span",{'class': 'team-country-name'}).get_text()

In [253]:
def get_seasons(link,i):
    print(link)
    print(i)
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    divs=bs.findAll("div",{'class': 'carousel-item'})
    for data in divs:
        Group[i].append(data.find("p").get_text().split("L")[0])
        for a in data.findAll("a",{'class':'table_team-name_block'}):
            country[i].append(get_nation(a.get("href")) )
        for span in data.findAll("span",{'class':'team-name js-team-name'}):
            teams[i].append(span.get_text().split("Playing now")[0])
    

In [254]:
for i in range(0,8):
    get_seasons(creat_seasons_link(i),i)

https://www.uefa.com/uefachampionsleague/history/seasons/2021/
0
https://www.uefa.com/uefachampionsleague/history/seasons/2020/
1
https://www.uefa.com/uefachampionsleague/history/seasons/2019/
2
https://www.uefa.com/uefachampionsleague/history/seasons/2018/
3
https://www.uefa.com/uefachampionsleague/history/seasons/2017/
4
https://www.uefa.com/uefachampionsleague/history/seasons/2016/
5
https://www.uefa.com/uefachampionsleague/history/seasons/2015/
6
https://www.uefa.com/uefachampionsleague/history/seasons/2014/
7


In [255]:
def retest(text):
    text=text.replace(".","")
    return re.sub(r'[^A-Za-z0-9]'," ",text).strip()

In [256]:
for n in range(0,8):
    for i in range(0,len(Group[n])):
        Group[n][i]=retest(Group[n][i])

    for i in range(0,len(teams[n])):
        teams[n][i]=retest(teams[n][i])

    for i in range(0,len(country[n])):
        country[n][i]=retest(country[n][i]) 


In [260]:
#country[1]
len(teams)

10

In [261]:
spans_team=[[] for _ in range(10)]
spans_HS=[[] for _ in range(10)]
spans_AS=[[] for _ in range(10)]
match_count=[[] for _ in range(10)]

In [262]:
for n in range(0,8):
    match_count[n].append( int (len(teams[n])/4*12) ) 
    team_count= int( len(teams[n])/4*2 )
    while team_count!=2:
        match_count[n][0]=match_count[n][0]+team_count
        team_count=team_count/2
    match_count[n][0]= int (match_count[n][0]+1)
    

In [264]:
def creat_match_link(year,indx):
    return ("https://www.uefa.com/uefachampionsleague/history/seasons/"
           +year
           +"/matches/libraries/matches-by-team/index_"
           +str(indx))

In [265]:
def get_Data(link,n):
    #print(link)
    html = urlopen(link)
    bs = BeautifulSoup(html, 'html.parser')
    spans_team[n].extend(bs.findAll("span",{'class': 'js-fitty'}))
    spans_HS[n].extend(bs.findAll("span",{'class': 'js-team--home-score home-score'}))
    spans_AS[n].extend(bs.findAll("span",{'class': 'js-team--away-score away-score'}))  

In [266]:
for n in range(0,8):
    year=2021-n
    if year<=2007:
        year=year-1
    for i in range(0,3):
        get_Data(creat_match_link(str(year),i),n)    


In [267]:
match=[[] for _ in range(10)]
n=0

In [268]:
for x in range(0,8):
    n=0
    for i in range(0, int(match_count[x][0]) ):
        if int(spans_HS[x][i].get_text())>int(spans_AS[x][i].get_text()):
            match[x].append(retest(spans_team[x][n].get_text()) 
                 +"# #"
                 +retest(spans_HS[x][i].get_text())
                 +"-"
                 +retest(spans_AS[x][i].get_text())
                 +"# #"
                 +retest(spans_team[x][n+1].get_text())
                 +"# #"+retest(spans_team[x][n].get_text()) )
        elif int(spans_HS[x][i].get_text())<int(spans_AS[x][i].get_text()):
            match[x].append(retest(spans_team[x][n].get_text()) 
                 +"# #"
                 +retest(spans_HS[x][i].get_text())
                 +"-"
                 +retest(spans_AS[x][i].get_text())
                 +"# #"
                 +retest(spans_team[x][n+1].get_text())
                 +"# #"+retest(spans_team[x][n+1].get_text()) )
        elif int(spans_HS[x][i].get_text())==int(spans_AS[x][i].get_text()):
            match[x].append(retest(spans_team[x][n].get_text()) 
                 +"# #"
                 +retest(spans_HS[x][i].get_text())
                 +"-"
                 +retest(spans_AS[x][i].get_text())
                 +"# #"
                 +retest(spans_team[x][n+1].get_text())
                 +"# #"+" X" )
        n=n+2 

In [299]:
match[8]

[]

In [275]:
#db.delete_all()

In [329]:
db=neo.Graph("neo4j+s://52d54f1d.databases.neo4j.io",
    auth=("neo4j","cUQmozgEst1IHWWMmbpPgPiRExkAYWk-qSZdKYcALwA"))

In [301]:
def create_node(team_nodes,country_nodes,type):
    for n in range(0,len(team_nodes)):
        if check_teams(team_nodes[n])==0:
            db.create(neo.Node(type,name=team_nodes[n],country=country_nodes[n]))

def check_teams(name_node):
    node_matcher = neo.NodeMatcher(db)
    node=node_matcher.match("Teams",name=name_node)
    node=list(node)
    if len(node):
        return 1
    else:
        return 0


for i in range(0,8):
    create_node(teams[i],country[i],"Teams")

In [302]:
def create_league(n):
    year=2021-n
    db.create(neo.Node("Champion league",year=year))

for n in range(0,8):
    create_league(n)

In [330]:
node_matcher = neo.NodeMatcher(db)
nodes_Teams=node_matcher.match("Teams")
nodes_Teams=list(nodes_Teams)
nodes_Cleagues=node_matcher.match("Champion league")
nodes_Cleagues=list(nodes_Cleagues)

In [304]:
R_participate=neo.Relationship.type("participate")
for n in range(0,len(nodes_Cleagues)):
    for T_node in nodes_Teams:
        if T_node["name"] in teams[n]:
            db.create(R_participate(nodes_Cleagues[n],T_node))

In [305]:
R_match=neo.Relationship.type("match")
for n in range(0,len(nodes_Cleagues)):
    for M in match[n]:
        GD= abs(int(M.split("# #")[1].split("-")[0])-int(M.split("# #")[1].split("-")[1]))
        proper={"general":M,'score': M.split("# #")[1],"winner":M.split("# #")[3],"GD":	GD}
        for node in nodes_Teams:
            if node["name"]==M.split("# #")[0]:
                H_T=node
            elif node["name"]==M.split("# #")[2]:
                A_T=node
        db.create(R_match(H_T,A_T,**proper))

In [343]:
for n in range(0,len(nodes_Cleagues)):
    for M in range(0,len(match[n])):
        if M==0:
            nodes_Cleagues[n]["champion"]=match[n][M].split("# #")[3]
            db.push(nodes_Cleagues[n])
        elif M>0 and M<=4:
            nodes_Cleagues[n]["Semi-finals"]=nodes_Cleagues[n]["Semi-finals"]+"-"+match[n][M].split("# #")[0]
            db.push(nodes_Cleagues[n]) 
        elif M>4 and M<=12:
            nodes_Cleagues[n]["Quarter-finals"]=nodes_Cleagues[n]["Quarter-finals"]+"-"+match[n][M].split("# #")[0]
            db.push(nodes_Cleagues[n])
        elif M>12 and M<=28:
            nodes_Cleagues[n]["Round of 16"]=nodes_Cleagues[n]["Round of 16"]+"-"+match[n][M].split("# #")[0]
            db.push(nodes_Cleagues[n])
            
nodes_Cleagues[5]["champion"]="Real Madrid"
db.push(nodes_Cleagues[5])


In [342]:
for n in range(0,len(nodes_Cleagues)):
    nodes_Cleagues[n]["Semi-finals"]=""
    nodes_Cleagues[n]["Quarter-finals"]=""
    nodes_Cleagues[n]["Round of 16"]=""
    db.push(nodes_Cleagues[n])

In [308]:
r_matcher = neo.RelationshipMatcher(db)

In [111]:
#R_1

In [84]:
#R_1[0].start_node['name'],R_1[0].end_node['year']

In [316]:
nodes_Teams[40]

Node('Teams', country='GRE', name='Olympiacos')

In [318]:
R_1=r_matcher.match([nodes_Teams[40]],r_type="match")
R_1=list(R_1)

In [321]:
R_1[0]

match(Node('Teams', country='GRE', name='Olympiacos'), Node('Teams', country='FRA', name='Paris'), GD=3, general='Olympiacos# #1-4# #Paris# #Paris', score='1-4', winner='Paris')